In [123]:
!gdown --id 1xJ8sahT95AdSDgr_immfnHmk1-VCpkAj
!gdown --id 1dTE9xqu5Sae3FIa8TIWJyvQ6FOv_HrRz

Downloading...
From: https://drive.google.com/uc?id=1xJ8sahT95AdSDgr_immfnHmk1-VCpkAj
To: /content/Supervisedready.csv
100% 2.06M/2.06M [00:00<00:00, 32.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dTE9xqu5Sae3FIa8TIWJyvQ6FOv_HrRz
To: /content/public-test.csv
100% 682k/682k [00:00<00:00, 10.7MB/s]


In [124]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [125]:
test=pd.read_csv('public-test.csv',delimiter='|')
test.head(1)
yTest = test['price']

In [126]:
df = pd.read_csv("Supervisedready.csv")
df

,Unnamed: 0,bookformat,bookedition,pages,weight,rating_value_0,rating_value_1,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,totalRating,weightRating1,weightRating0,weightKategori,volume,weight_per_page,lexile_measureNumber,400,401,402,403,404,500,501,502,503,600,601,602,603,604,1,703,704,705,price
0,9,5,-1,504.000013,3628.739803,4.29,4.6,7.000000,26.999999,19.000000,219.000007,3.000000,196.000007,70.999998,100.999998,11.000000,296.000007,0.26,0.21,0.17,0.10,0.08,0.05,0.04,0.04,0.03,0.02,27486.999383,105.999998,204.999996,304.0,593.963024,7.199882,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,98172.000000
1,17,3,-1,356.999975,1760.554250,4.89,0.0,999.000015,999.000015,999.000015,999.000015,999.000015,999.000106,999.000106,999.000106,999.000015,999.000015,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,27.999998,99.999997,205.999998,303.0,1643.900070,4.931523,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,57604.000000
2,29,7,-1,324.000002,3991.609957,3.99,4.6,15.000000,5.000000,91.999999,67.999998,29.000000,62.000001,100.999998,9.999999,6.000000,58.000000,0.45,0.19,0.07,0.07,0.06,0.05,0.04,0.04,0.03,0.02,45193.999479,105.999998,204.999996,304.0,497.887154,12.319783,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,103658.000000
3,33,2,-1,528.000019,771.109986,3.68,4.3,3.000000,5.000000,99.000002,221.000005,81.999997,53.000003,171.999998,18.999999,101.000001,91.999999,0.77,0.07,0.05,0.02,0.02,0.02,0.02,0.01,0.01,0.01,20886.000318,105.000002,204.999996,301.0,1742.425816,1.460436,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,649665.000000
4,44,7,-1,499.999988,725.749981,3.81,3.6,999.000015,999.000015,999.000015,999.000015,999.000015,999.000106,999.000106,999.000106,999.000015,999.000015,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,40.999999,105.000002,204.999996,301.0,1104.384885,1.451500,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,247883.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3544,3545,7,53,448.000007,1215.295654,4.19,0.0,19.000000,57.000000,29.000000,10.000000,77.000000,244.000007,5.000000,31.000001,328.999987,201.000002,0.36,0.24,0.10,0.07,0.07,0.06,0.03,0.02,0.02,0.02,172198.996672,99.999997,204.999996,302.0,1816.875888,2.712714,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,335101.949308
3545,3546,2,-1,477.999986,1041.068067,4.21,0.0,3.000000,24.000001,101.000001,5.000000,11.000000,26.999999,194.000006,91.999999,58.000000,10.000000,0.43,0.18,0.11,0.11,0.04,0.04,0.03,0.02,0.02,0.02,43150.001363,99.999997,204.999996,302.0,1856.574269,2.177967,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,323153.302721
3546,3547,2,26,352.000014,1758.964606,3.55,0.0,8.000000,80.999998,2.000000,26.000000,256.000000,999.000015,999.000015,999.000015,999.000015,999.000015,0.68,0.14,0.13,0.05,0.01,0.00,0.00,0.00,0.00,0.00,5811.999654,99.999997,204.999996,303.0,1719.444322,4.997059,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,305593.965332
3547,3548,3,-1,348.999987,1800.329552,4.64,0.0,36.000001,999.000015,999.000015,999.000015,999.000015,999.000015,999.000015,999.000015,999.000015,999.000015,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15.000000,99.999997,205.999998,303.0,1647.731667,5.158538,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,197921.433935


In [127]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

In [128]:
X = df.drop("price",axis=1)
Y = df["price"]

numerics = [
            "pages"	,"weight","rating_value_0"
            ,"rating_value_1",'genre_0_weight',	'genre_1_weight',	'genre_2_weight'	
            ,'genre_3_weight'	,'genre_4_weight'	,'genre_5_weight',	'genre_6_weight'	
            ,'genre_7_weight',	"genre_8_weight",	"genre_9_weight",'totalRating'	,'weightRating1'	,'weightRating0'	
            ,'weightKategori'	,'volume'	,"weight_per_page","lexile_measureNumber",
            "genre_0","genre_1","genre_2","genre_3","genre_4","genre_5","genre_6","genre_7","genre_8","genre_9","lexile_measureNumber"
]

scalerX = MinMaxScaler() 
scalerY = MinMaxScaler()

X[numerics] = scalerX.fit_transform(X[numerics])
Y = scalerY.fit_transform(Y.values.reshape(-1,1)) 

In [129]:
from sklearn.metrics import mean_squared_error

In [130]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(X,Y.reshape(len(Y),),random_state=2,test_size=0.05)

RandomForest = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=4, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

RandomForest.fit(x_train,y_train)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=4, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [131]:
print(mean_squared_error(RandomForest.predict(x_val),y_val))

0.00181062306251658


In [132]:
import tensorflow as tf
import math
from tensorflow.keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 


Percentage = 0.3
input = tf.keras.layers.Input(shape=(x_train.shape[1]))

layers = tf.keras.layers.Dense(units=1024,activation=tf.nn.relu)(input)
layers = tf.keras.layers.Dropout(Percentage + 0.5 )(layers)
layers = tf.keras.layers.Dense(units=128,activation=tf.nn.relu)(layers)
layers = tf.keras.layers.Dense(units=128,activation=tf.nn.relu)(layers)
layers = tf.keras.layers.Dense(units=52,activation=tf.nn.relu)(layers)
layers = tf.keras.layers.Dropout(Percentage )(layers)
layers = tf.keras.layers.Dense(units=1)(layers)
model = tf.keras.Model(inputs=[input],outputs=[layers])
model.compile(loss=root_mean_squared_error,optimizer=tf.keras.optimizers.Adam())
model.summary()

def create_learning_rate_scheduler(max_learn_rate=1e-3,
                                   end_learn_rate=1e-4,
                                   warmup_epoch_count=150,
                                   total_epoch_count=200):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = max_learn_rate
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 52)]              0         
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              54272     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               131200    
_________________________________________________________________
dense_17 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_18 (Dense)             (None, 52)                6708      
_________________________________________________________________
dropout_7 (Dropout)          (None, 52)                0   

In [133]:
model.fit(x_train,y_train,epochs=200,
          callbacks=[create_learning_rate_scheduler()],verbose=0)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00004: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00006: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00007: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00008: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00009: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00010: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00011: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00012: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00013: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00014: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00015: LearningRateSchedule

In [134]:
print(mean_squared_error(model.predict(x_val),y_val))

0.0046438944936184464


In [135]:
test.isna().sum()

author_id           0
description         8
bookformat          0
bookedition       446
pages              20
published_date     31
publisher_id       31
reading_age       381
lexile_measure    403
grade_level       405
weight             62
rating_value_0      0
rating_value_1     56
rating_count_0      0
rating_count_1      0
dimension_0        63
dimension_1        63
dimension_2        81
genre_0            41
genre_1            47
genre_2            55
genre_3            63
genre_4            68
genre_5            73
genre_6            77
genre_7            80
genre_8            82
genre_9            87
genre_0_weight     41
genre_1_weight     47
genre_2_weight     55
genre_3_weight     63
genre_4_weight     68
genre_5_weight     73
genre_6_weight     77
genre_7_weight     80
genre_8_weight     82
genre_9_weight     87
price               0
dtype: int64

In [136]:
eksplor=test[["description",'reading_age','lexile_measure','grade_level']]

In [137]:

test['author_id'] = test['author_id'].astype('category')
test['publisher_id'] = test['publisher_id'].astype('category')
test['bookformat'] = test['bookformat'].astype('category')
test['bookedition'] = test['bookedition'].astype('category')


test['author_id']=test['author_id'].cat.codes
test['publisher_id']=test['publisher_id'].cat.codes
test['bookformat']=test['bookformat'].cat.codes
test['bookedition']=test['bookedition'].cat.codes

test.head(2)

,author_id,description,bookformat,bookedition,pages,published_date,publisher_id,reading_age,lexile_measure,grade_level,weight,rating_value_0,rating_value_1,rating_count_0,rating_count_1,dimension_0,dimension_1,dimension_2,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,price
0,359,Rachel Friedman has always been the consummate...,4,-1,295.0,"March 29, 2011",14,NaN,NaN,NaN,3628.74,3.79,4.4,3412,178,13.21,1.78,20.32,Travel,Nonfiction,Memoir,Adventure,Biography,Biography Memoir,Chick Lit,Adult,Autobiography,Contemporary,0.53,0.20,0.14,0.03,0.03,0.02,0.02,0.01,0.01,0.01,129789.0
1,32,As Dr. Marina Singh embarks upon an uncertain ...,1,-1,353.0,"June 7, 2011",72,NaN,990,NaN,521.63,3.88,4.1,168718,2908,15.24,2.97,22.86,Fiction,Contemporary,Book Club,Literary Fiction,Adult Fiction,Adult,Mystery,Audiobook,Adventure,Novels,0.62,0.06,0.05,0.05,0.05,0.04,0.04,0.03,0.03,0.03,262465.0


In [138]:
res=pd.Series()
for genre in test.columns[18:28]:
    res=res.append(test[genre])

res=pd.Series(res.unique())
dicGenre=res.to_dict()
dicGenre = dict((v,k) for k,v in dicGenre.items())
dicGenre[np.nan]=999
dicGenre


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


{'12th Century': 225,
 '15th Century': 310,
 '16th Century': 319,
 '18th Century': 181,
 '19th Century': 208,
 '1st Grade': 300,
 '21st Century': 299,
 'Abuse': 210,
 'Action': 96,
 'Activism': 284,
 'Adult': 65,
 'Adult Fiction': 49,
 'Adventure': 36,
 'Africa': 111,
 'African American': 211,
 'Aliens': 131,
 'Alternate History': 238,
 'Amateur Sleuth': 257,
 'Amazon': 296,
 'American': 221,
 'American History': 142,
 'American Revolution': 303,
 'Americana': 323,
 'Angels': 112,
 'Anglo Saxon': 324,
 'Animal Fiction': 302,
 'Animals': 55,
 'Anthologies': 95,
 'Anthropomorphic': 326,
 'Apocalyptic': 119,
 'Art': 152,
 'Arthurian': 155,
 'Asia': 117,
 'Asian Literature': 228,
 'Atheism': 193,
 'Audiobook': 102,
 'Australia': 134,
 'Autobiography': 166,
 'Aviation': 292,
 'BDSM': 69,
 'Bande Dessinée': 325,
 'Baseball': 114,
 'Biography': 32,
 'Biography Memoir': 135,
 'Biology': 133,
 'Birds': 294,
 'Book Club': 93,
 'Books About Books': 105,
 'Brain': 283,
 'Brazil': 290,
 'British Li

In [139]:
for genre in test.columns[18:28]:
    test[genre]=test[genre].replace(dicGenre)

test

,author_id,description,bookformat,bookedition,pages,published_date,publisher_id,reading_age,lexile_measure,grade_level,weight,rating_value_0,rating_value_1,rating_count_0,rating_count_1,dimension_0,dimension_1,dimension_2,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,price
0,359,Rachel Friedman has always been the consummate...,4,-1,295.0,"March 29, 2011",14,NaN,NaN,NaN,3628.74,3.79,4.4,3412,178,13.21,1.78,20.32,0,9,4,36,32,135,62,65,166,28,0.53,0.20,0.14,0.03,0.03,0.02,0.02,0.01,0.01,0.01,129789.0
1,32,As Dr. Marina Singh embarks upon an uncertain ...,1,-1,353.0,"June 7, 2011",72,NaN,990,NaN,521.63,3.88,4.1,168718,2908,15.24,2.97,22.86,1,28,93,81,49,65,3,102,36,84,0.62,0.06,0.05,0.05,0.05,0.04,0.04,0.03,0.03,0.03,262465.0
2,357,From the moment she took a job on Captain Cald...,4,20,373.0,"April 22, 2014",125,NaN,NaN,NaN,5443.10,3.96,4.5,6845,304,13.97,2.54,20.95,2,19,75,131,1,144,12,65,36,126,0.44,0.16,0.11,0.06,0.06,0.05,0.04,0.04,0.02,0.02,182195.0
3,280,"#1 New York Times bestseller Lisa Gardner, aut...",1,-1,423.0,"February 5, 2013",46,NaN,NaN,NaN,680.39,4.07,4.6,30037,1887,16.51,3.68,23.75,3,14,30,1,64,35,102,49,65,264,0.30,0.18,0.14,0.13,0.08,0.07,0.04,0.02,0.02,0.02,288596.0
4,276,This is not your mother’s memoir. In The Chron...,4,-1,310.0,"April 1, 2011",79,NaN,NaN,NaN,453.59,4.23,4.4,9193,463,14.22,2.03,22.61,4,9,54,32,138,43,135,166,121,273,0.44,0.24,0.06,0.06,0.05,0.04,0.03,0.03,0.02,0.01,230270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,3,World War I stands as one of history's most se...,1,-1,448.0,"May 3, 2011",83,NaN,NaN,NaN,771.11,4.15,4.6,7953,409,15.88,4.45,23.50,20,9,130,97,177,25,126,235,240,56,0.50,0.22,0.09,0.07,0.02,0.02,0.02,0.02,0.02,0.01,352263.0
496,24,"A sweeping, evocative epic of two women's inte...",1,-1,589.0,"November 5, 2013",49,NaN,NaN,NaN,748.43,3.65,3.8,39510,2920,15.24,3.73,22.86,8,1,66,56,117,228,65,49,102,84,0.36,0.33,0.10,0.06,0.04,0.02,0.02,0.02,0.02,0.02,307364.0
497,88,She could remember standing in a park near the...,1,-1,38.0,"April 4, 2011",67,6 - 9 years,1060L,1 - 4,7121.39,3.90,4.6,4330,103,20.32,1.17,29.21,22,32,9,58,20,36,135,56,121,244,0.35,0.21,0.14,0.09,0.08,0.05,0.03,0.03,0.02,0.02,274159.0
498,337,"Following on the success of Tender and Ripe, t...",1,-1,544.0,"September 24, 2013",176,NaN,NaN,NaN,1564.89,4.31,4.5,799,36,17.78,4.80,24.71,26,92,9,40,47,229,91,4,186,328,0.39,0.19,0.13,0.12,0.05,0.03,0.03,0.03,0.03,0.03,489992.0


In [140]:
for genre in test.columns[28:38]:
    test[genre]=test[genre].fillna(0)


test.head(20)

,author_id,description,bookformat,bookedition,pages,published_date,publisher_id,reading_age,lexile_measure,grade_level,weight,rating_value_0,rating_value_1,rating_count_0,rating_count_1,dimension_0,dimension_1,dimension_2,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,price
0,359,Rachel Friedman has always been the consummate...,4,-1,295.0,"March 29, 2011",14,NaN,NaN,NaN,3628.74,3.79,4.4,3412,178,13.21,1.78,20.32,0,9,4,36,32,135,62,65,166,28,0.53,0.20,0.14,0.03,0.03,0.02,0.02,0.01,0.01,0.01,129789.0
1,32,As Dr. Marina Singh embarks upon an uncertain ...,1,-1,353.0,"June 7, 2011",72,NaN,990,NaN,521.63,3.88,4.1,168718,2908,15.24,2.97,22.86,1,28,93,81,49,65,3,102,36,84,0.62,0.06,0.05,0.05,0.05,0.04,0.04,0.03,0.03,0.03,262465.0
2,357,From the moment she took a job on Captain Cald...,4,20,373.0,"April 22, 2014",125,NaN,NaN,NaN,5443.10,3.96,4.5,6845,304,13.97,2.54,20.95,2,19,75,131,1,144,12,65,36,126,0.44,0.16,0.11,0.06,0.06,0.05,0.04,0.04,0.02,0.02,182195.0
3,280,"#1 New York Times bestseller Lisa Gardner, aut...",1,-1,423.0,"February 5, 2013",46,NaN,NaN,NaN,680.39,4.07,4.6,30037,1887,16.51,3.68,23.75,3,14,30,1,64,35,102,49,65,264,0.30,0.18,0.14,0.13,0.08,0.07,0.04,0.02,0.02,0.02,288596.0
4,276,This is not your mother’s memoir. In The Chron...,4,-1,310.0,"April 1, 2011",79,NaN,NaN,NaN,453.59,4.23,4.4,9193,463,14.22,2.03,22.61,4,9,54,32,138,43,135,166,121,273,0.44,0.24,0.06,0.06,0.05,0.04,0.03,0.03,0.02,0.01,230270.0
5,47,What if the world's worst serial killer...was ...,1,-1,359.0,"April 3, 2012",103,15 years and up,HL750L,10 and up,471.74,4.02,4.6,39110,627,15.24,3.17,22.23,5,3,14,11,35,1,28,64,30,199,0.26,0.22,0.13,0.12,0.08,0.05,0.04,0.04,0.03,0.03,144081.0
6,80,"After his December 2003 arrest, registered nur...",1,-1,307.0,"April 15, 2013",183,NaN,NaN,NaN,576.06,3.81,4.5,15470,1310,15.88,2.54,23.50,6,9,35,132,32,102,77,3,20,307,0.37,0.35,0.08,0.05,0.04,0.04,0.03,0.02,0.02,0.01,385324.0
7,204,She's writing about him. He's writing about he...,4,-1,243.0,"July 19, 2011",108,14 - 17 years,NaN,8 - 12,3628.74,3.61,3.8,13281,131,13.49,2.03,20.95,5,19,28,59,62,201,1,52,63,272,0.29,0.25,0.14,0.09,0.06,0.04,0.03,0.03,0.03,0.03,215400.0
8,180,Some people have everything figured out—Andrea...,3,-1,434.0,"July 31, 2012",2,NaN,NaN,NaN,3265.86,4.24,4.7,34443,893,10.67,3.05,17.02,7,12,15,19,73,21,206,44,1,65,0.26,0.19,0.14,0.10,0.08,0.06,0.05,0.05,0.03,0.03,115352.0
9,290,A powerful and heartbreaking WWII historical n...,4,-1,384.0,"June 22, 2021",68,NaN,NaN,NaN,4309.12,3.90,4.0,619,992,12.90,19.81,NaN,8,1,94,97,49,65,84,81,149,308,0.47,0.11,0.09,0.06,0.06,0.04,0.04,0.04,0.04,0.04,209192.0


In [141]:
test.loc[test['rating_count_1'] <= 1, 'rating_value_1']=0
test.loc[test['rating_count_0'] <= 1, 'rating_value_0']=0

In [142]:
dataImpute=test[['pages','weight','dimension_0','dimension_1','dimension_2','rating_value_0','rating_value_1']]

dataImpute[90:100]

,pages,weight,dimension_0,dimension_1,dimension_2,rating_value_0,rating_value_1
90,290.0,NaN,NaN,NaN,NaN,4.00,0.0
91,304.0,789.25,15.54,23.50,NaN,4.26,0.0
92,288.0,5080.23,13.97,3.81,20.95,3.85,4.5
93,400.0,5579.18,13.97,2.87,20.95,4.06,4.3
94,531.0,725.75,15.24,4.32,22.86,4.32,4.6
95,184.0,625.96,17.68,1.85,26.75,3.92,3.9
96,294.0,462.66,13.97,20.95,NaN,4.33,4.7
97,256.0,453.59,15.24,2.39,22.86,3.90,4.5
98,272.0,NaN,NaN,NaN,NaN,4.25,4.7
99,272.0,476.27,16.51,2.54,24.13,3.31,4.1


In [143]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Q1=int(test['weight'].quantile(0.25))
Q3=int(test['weight'].quantile(0.75))
IQR=Q3-Q1
batasatas=int(Q3+1.5*IQR)
batasbawah=int(Q1-1.5*IQR)

imp = IterativeImputer(max_iter=50, random_state=3,initial_strategy='mean',max_value=batasatas,min_value=batasbawah)
imp.fit(dataImpute)
miceImpute=imp.transform(dataImpute)
pd.DataFrame(miceImpute)[90:100]

,0,1,2,3,4,5,6
90,290.0,1623.997883,14.788871,8.414154,18.512860,4.00,0.0
91,304.0,789.250000,15.540000,23.500000,10.195167,4.26,0.0
92,288.0,5080.230000,13.970000,3.810000,20.950000,3.85,4.5
93,400.0,5579.180000,13.970000,2.870000,20.950000,4.06,4.3
94,531.0,725.750000,15.240000,4.320000,22.860000,4.32,4.6
95,184.0,625.960000,17.680000,1.850000,26.750000,3.92,3.9
96,294.0,462.660000,13.970000,20.950000,12.574602,4.33,4.7
97,256.0,453.590000,15.240000,2.390000,22.860000,3.90,4.5
98,272.0,2903.699478,15.899737,3.016885,22.278454,4.25,4.7
99,272.0,476.270000,16.510000,2.540000,24.130000,3.31,4.1


In [144]:
hasil=pd.DataFrame(miceImpute)
hasil=hasil[[0,1,2,3,4]]
test[['pages','weight','dimension_0','dimension_1','dimension_2']]=hasil
test['pages'] = test['pages'].astype(int)
test

,author_id,description,bookformat,bookedition,pages,published_date,publisher_id,reading_age,lexile_measure,grade_level,weight,rating_value_0,rating_value_1,rating_count_0,rating_count_1,dimension_0,dimension_1,dimension_2,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,price
0,359,Rachel Friedman has always been the consummate...,4,-1,295,"March 29, 2011",14,NaN,NaN,NaN,3628.74,3.79,4.4,3412,178,13.21,1.78,20.32,0,9,4,36,32,135,62,65,166,28,0.53,0.20,0.14,0.03,0.03,0.02,0.02,0.01,0.01,0.01,129789.0
1,32,As Dr. Marina Singh embarks upon an uncertain ...,1,-1,353,"June 7, 2011",72,NaN,990,NaN,521.63,3.88,4.1,168718,2908,15.24,2.97,22.86,1,28,93,81,49,65,3,102,36,84,0.62,0.06,0.05,0.05,0.05,0.04,0.04,0.03,0.03,0.03,262465.0
2,357,From the moment she took a job on Captain Cald...,4,20,373,"April 22, 2014",125,NaN,NaN,NaN,5443.10,3.96,4.5,6845,304,13.97,2.54,20.95,2,19,75,131,1,144,12,65,36,126,0.44,0.16,0.11,0.06,0.06,0.05,0.04,0.04,0.02,0.02,182195.0
3,280,"#1 New York Times bestseller Lisa Gardner, aut...",1,-1,423,"February 5, 2013",46,NaN,NaN,NaN,680.39,4.07,4.6,30037,1887,16.51,3.68,23.75,3,14,30,1,64,35,102,49,65,264,0.30,0.18,0.14,0.13,0.08,0.07,0.04,0.02,0.02,0.02,288596.0
4,276,This is not your mother’s memoir. In The Chron...,4,-1,310,"April 1, 2011",79,NaN,NaN,NaN,453.59,4.23,4.4,9193,463,14.22,2.03,22.61,4,9,54,32,138,43,135,166,121,273,0.44,0.24,0.06,0.06,0.05,0.04,0.03,0.03,0.02,0.01,230270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,3,World War I stands as one of history's most se...,1,-1,448,"May 3, 2011",83,NaN,NaN,NaN,771.11,4.15,4.6,7953,409,15.88,4.45,23.50,20,9,130,97,177,25,126,235,240,56,0.50,0.22,0.09,0.07,0.02,0.02,0.02,0.02,0.02,0.01,352263.0
496,24,"A sweeping, evocative epic of two women's inte...",1,-1,589,"November 5, 2013",49,NaN,NaN,NaN,748.43,3.65,3.8,39510,2920,15.24,3.73,22.86,8,1,66,56,117,228,65,49,102,84,0.36,0.33,0.10,0.06,0.04,0.02,0.02,0.02,0.02,0.02,307364.0
497,88,She could remember standing in a park near the...,1,-1,38,"April 4, 2011",67,6 - 9 years,1060L,1 - 4,7121.39,3.90,4.6,4330,103,20.32,1.17,29.21,22,32,9,58,20,36,135,56,121,244,0.35,0.21,0.14,0.09,0.08,0.05,0.03,0.03,0.02,0.02,274159.0
498,337,"Following on the success of Tender and Ripe, t...",1,-1,544,"September 24, 2013",176,NaN,NaN,NaN,1564.89,4.31,4.5,799,36,17.78,4.80,24.71,26,92,9,40,47,229,91,4,186,328,0.39,0.19,0.13,0.12,0.05,0.03,0.03,0.03,0.03,0.03,489992.0


In [145]:
test=test.drop(["description",'published_date','reading_age','lexile_measure','grade_level'],axis=1)

In [146]:
test['weight']  = np.where(test["weight"] < 0, test["weight"] * 0,test['weight'] * 1)

In [147]:
test['totalRating'] = test["rating_count_0"] + test["rating_count_1"]
test.drop(["rating_count_0","rating_count_1"],axis=1,inplace=True)

In [148]:
test["rating_value_1"].unique()

condition = [
             test["rating_value_1"] == 0,
             (test["rating_value_1"] > 0) & (test["rating_value_1"]   < 1),
             (test["rating_value_1"] > 1) & (test["rating_value_1"]   < 1.5 ),
             (test["rating_value_1"] > 1.5) & (test["rating_value_1"] < 2.5 ),
             (test["rating_value_1"] > 2.5) & (test["rating_value_1"] < 3.5),
             (test["rating_value_1"] > 3.5) & (test["rating_value_1"] < 4.5),
             (test["rating_value_1"] > 4.5)  
]
condition2 = [
             test["rating_value_0"] == 0,
             (test["rating_value_0"] > 0) & (test["rating_value_0"]   < 1),
             (test["rating_value_0"] > 1) & (test["rating_value_0"]   < 1.5 ),
             (test["rating_value_0"] > 1.5) & (test["rating_value_0"] < 2.5 ),
             (test["rating_value_0"] > 2.5) & (test["rating_value_0"] < 3.5),
             (test["rating_value_0"] > 3.5) & (test["rating_value_0"] < 4.5),
             (test["rating_value_0"] > 4.5)  
]

result = [
          "tidak",
          "sangat buruk",
          "buruk",
          "cukup",
          "lumayan",
          "baik",
          "sangat baik"
]
weightRating1 = np.select(condition,result)
test["weightRating1"] = weightRating1
test["weightRating1"] = test["weightRating1"].replace({"tidak" : 100,
                                                           "sangat buruk" : 101,
                                                           "buruk" : 102,
                                                           "cukup" : 103,
                                                           "lumayan" : 104,
                                                           "baik" : 105,
                                                         "sangat baik" : 106})
weightRating0 = np.select(condition2,result)
test["weightRating0"] = weightRating0
test["weightRating0"] = test["weightRating0"].replace({"tidak" : 200,
                                                           "sangat buruk" : 201,
                                                           "buruk" : 202,
                                                           "cukup" : 203,
                                                           "lumayan" : 204,
                                                           "baik" : 205,
                                                           "sangat baik" : 206})

In [149]:
condition = [
             test["weight"] < 500,
             (test["weight"] > 500) & (test["weight"] <1000),
             (test["weight"] > 1000) & (test["weight"] < 1500),
             (test["weight"] > 1500) & (test["weight"] < 3000),
             (test["weight"] > 3000)
]
result = [
          "sangat ringan",
          "ringan",
          "sedang",
          "berat",
          "sangat berat"
]
weightKategori = np.select(condition,result)
test["weightKategori"] = weightKategori
test["weightKategori"] = test["weightKategori"].replace({"sangat ringan" : 300,
                                                           "ringan" : 301,
                                                           "sedang" : 302,
                                                           "berat" : 303,
                                                           "sangat berat" : 304})

In [150]:
test["volume"] = test["dimension_0"] * test["dimension_1"] * test["dimension_2"]
test.drop(["dimension_0","dimension_1","dimension_2"],axis= 1, inplace=True)
test.head(1)

,author_id,bookformat,bookedition,pages,publisher_id,weight,rating_value_0,rating_value_1,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,price,totalRating,weightRating1,weightRating0,weightKategori,volume
0,359,4,-1,295,14,3628.74,3.79,4.4,0,9,4,36,32,135,62,65,166,28,0.53,0.2,0.14,0.03,0.03,0.02,0.02,0.01,0.01,0.01,129789.0,3590,105,205,304,477.800416


In [151]:
test["weight_per_page"] = test["weight"] / test["pages"]
test.head(1)

,author_id,bookformat,bookedition,pages,publisher_id,weight,rating_value_0,rating_value_1,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,price,totalRating,weightRating1,weightRating0,weightKategori,volume,weight_per_page
0,359,4,-1,295,14,3628.74,3.79,4.4,0,9,4,36,32,135,62,65,166,28,0.53,0.2,0.14,0.03,0.03,0.02,0.02,0.01,0.01,0.01,129789.0,3590,105,205,304,477.800416,12.300814


In [152]:
condition = [
             eksplor["lexile_measure"].str.match(r"AD\d+") == True,
             eksplor["lexile_measure"].str.match(r"HL\d+") == True,
             eksplor["lexile_measure"].str.match(r"GN\d+") == True,
             eksplor["lexile_measure"].str.match(r"\d+") == True,
             eksplor["lexile_measure"].str.match(r"NP") == True,
]
res = [
        "adult directed",
        "high low",
        "comic",
        "ordinary",
        "entertainment"
]

eksplor["lexile_codes"] = np.select(condition,res)
eksplor["lexile_codes"] = eksplor["lexile_codes"].replace({
    "0" : 400,
    "adult directed" : 401,
    "comic": 402,
    "high low" : 403,
    "ordinary" : 404,
    "entertainment" : 405
})
eksplor.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,description,reading_age,lexile_measure,grade_level,lexile_codes
0,Rachel Friedman has always been the consummate...,NaN,NaN,NaN,400


In [153]:
eksplor["lexile_measureNumber"] = eksplor["lexile_measure"].str.replace(r"[a-zA-Z]",'')

eksplor["lexile_measureNumber"].unique()
eksplor["lexile_measureNumber"].fillna("0",inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [154]:
eksplor[eksplor['lexile_measureNumber'] == ""] = 400

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [155]:
eksplor['lexile_measureNumber'] = eksplor["lexile_measureNumber"].astype('float')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [156]:
condition = [
             (eksplor["lexile_measureNumber"] > 190) & (eksplor["lexile_measureNumber"] < 925),
             (eksplor["lexile_measureNumber"] > 925 ) & (eksplor["lexile_measureNumber"] < 1050),
             (eksplor["lexile_measureNumber"] > 1050)
]

res = [
       "low",
       "sufficient",
       "proficient"
]

res = np.select(condition,res)
eksplor["lexile_categorical"] = res
eksplor["lexile_categorical"] = eksplor['lexile_categorical'].replace({
    "low" : 501,
    "sufficient" : 502,
    "proficient" : 503,
    "0" : 500
})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [157]:
condition  = [
              (eksplor["grade_level"] == "Preschool - Kindergarten") | (eksplor["grade_level"] == "Kindergarten - 3") |
              (eksplor["grade_level"] == "Preschool - 1") | (eksplor["grade_level"] == "Preschool - 2") |
              (eksplor["grade_level"] == "Preschool - 2") | (eksplor["grade_level"] == "1 - 2"),
              
              (eksplor["grade_level"] == "3 - 7") | (eksplor["grade_level"] == "4 - 6") |
              (eksplor["grade_level"] == "5 - 6") | (eksplor["grade_level"] == "4 - 7") ,


              (eksplor["grade_level"] == "5 - 8") | (eksplor["grade_level"] == "5 - 7") |
              (eksplor["grade_level"] == "5 - 9") | (eksplor["grade_level"] == "6 - 9") |
              (eksplor["grade_level"] == "6 - 9"), 


              (eksplor["grade_level"] == "8 and up") | (eksplor["grade_level"] == "9 and up") |
              (eksplor["grade_level"] == "7 and up") | (eksplor["grade_level"] == "10 and up") |
              (eksplor["grade_level"] == "8 - 12") | (eksplor["grade_level"] == "8 - 9") |
              (eksplor["grade_level"] == "9 - 12") | (eksplor["grade_level"] == "7 - 9") |
              (eksplor["grade_level"] == "10 - 12") | (eksplor["grade_level"] ==  "11 and up") |
              (eksplor["grade_level"] == "7 - 12")
]

res = [
       "very easy",
       "easy",
       "medium",
       "hard"
]

eksplor["grade_category"] = np.select(condition,res)
eksplor['grade_category']  = eksplor["grade_category"].replace({
    "0" : 600,
    "very easy" : 601,
    "easy" : 602,
    "medium" : 603,
    "hard" : 604
})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [158]:
condition = [ 
           ((eksplor['reading_age'] == "3 - 5 years") | (eksplor["reading_age"] == "4 - 8 years") |
           (eksplor["reading_age"] == "5 - 8 years") | (eksplor["reading_age"] == "3 - 7 years") |
           (eksplor["reading_age"] == "3 - 6 years") | (eksplor["reading_age"] == "4 - 6 years") |
           (eksplor["reading_age"] == "4 - 7 years") | (eksplor["reading_age"] == "3 - 8 years") |
           (eksplor["reading_age"] == "2 - 5 years")) ,

           ((eksplor["reading_age"] == "8 - 11 years") | (eksplor["reading_age"] == "8 - 12 years")) |
           ((eksplor["reading_age"] == "10 - 13 years") | (eksplor["reading_age"] == "9 - 12 years") | 
           (eksplor["reading_age"] == "10 - 12 years") | (eksplor["reading_age"] == "10 years") |
           (eksplor["reading_age"] == "10 years and up") | (eksplor["reading_age"] == "6 - 11 years")),

           ((eksplor["reading_age"] == "12 - 15 years") | (eksplor["reading_age"] == "13 years and up") |
           (eksplor["reading_age"] == "10 - 14 years") | (eksplor["reading_age"] == "12 years up") | 
           (eksplor["reading_age"] == "11 years up") | (eksplor["reading_age"] == "14 years and up") |
           (eksplor["reading_age"] == "15 years and up") | (eksplor["reading_age"] == "12 years and up"))

]

result =[
         "TK","SD","Others"
]
val = np.select(condition,result)
eksplor.loc[:,"reading_age_converted"] = val
eksplor['reading_age_converted'] = eksplor["reading_age_converted"].replace({"Others" : 703, "SD" : 704, "TK" : 1,"0" : 705})
eksplor.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,description,reading_age,lexile_measure,grade_level,lexile_codes,lexile_measureNumber,lexile_categorical,grade_category,reading_age_converted
0,Rachel Friedman has always been the consummate...,NaN,NaN,NaN,400,0.0,500,600,705
1,As Dr. Marina Singh embarks upon an uncertain ...,NaN,990,NaN,404,990.0,502,600,705
2,From the moment she took a job on Captain Cald...,NaN,NaN,NaN,400,0.0,500,600,705


In [159]:
test["lexile_codes"] = eksplor["lexile_codes"]
test["lexile_measureNumber"] = eksplor["lexile_measureNumber"]
test["lexile_categorical"] = eksplor["lexile_categorical"]
test["grade_category"] = eksplor["grade_category"]
test["reading_age_converted"] = eksplor["reading_age_converted"]

In [160]:
dummiesLexileCodes = pd.get_dummies(test["lexile_codes"])
dummiessLexileCategorical = pd.get_dummies(test["lexile_categorical"])
dummiesGradeCategory = pd.get_dummies(test["grade_category"])
dummiesReadingAge = pd.get_dummies(test["reading_age_converted"])


droppedCols = ["lexile_codes",
               "lexile_categorical","grade_category",
               "reading_age_converted"
               ]

test[dummiesLexileCodes.columns] = dummiesLexileCodes
test[dummiessLexileCategorical.columns] = dummiessLexileCategorical
test[dummiesGradeCategory.columns] = dummiesGradeCategory
test[dummiesReadingAge.columns] = dummiesReadingAge

test.drop(droppedCols,axis=1,inplace=True)

In [161]:
test.head(4)

,author_id,bookformat,bookedition,pages,publisher_id,weight,rating_value_0,rating_value_1,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_0_weight,genre_1_weight,genre_2_weight,genre_3_weight,genre_4_weight,genre_5_weight,genre_6_weight,genre_7_weight,genre_8_weight,genre_9_weight,price,totalRating,weightRating1,weightRating0,weightKategori,volume,weight_per_page,lexile_measureNumber,400,401,402,403,404,500,501,502,503,600,601,602,603,604,1,703,704,705
0,359,4,-1,295,14,3628.74,3.79,4.4,0,9,4,36,32,135,62,65,166,28,0.53,0.20,0.14,0.03,0.03,0.02,0.02,0.01,0.01,0.01,129789.0,3590,105,205,304,477.800416,12.300814,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
1,32,1,-1,353,72,521.63,3.88,4.1,1,28,93,81,49,65,3,102,36,84,0.62,0.06,0.05,0.05,0.05,0.04,0.04,0.03,0.03,0.03,262465.0,171626,105,205,301,1034.707608,1.477705,990.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1
2,357,4,20,373,125,5443.10,3.96,4.5,2,19,75,131,1,144,12,65,36,126,0.44,0.16,0.11,0.06,0.06,0.05,0.04,0.04,0.02,0.02,182195.0,7149,0,205,304,743.385610,14.592761,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
3,280,1,-1,423,46,680.39,4.07,4.6,3,14,30,1,64,35,102,49,65,264,0.30,0.18,0.14,0.13,0.08,0.07,0.04,0.02,0.02,0.02,288596.0,31924,106,205,301,1442.974000,1.608487,0.0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1


In [162]:
testcopy = test.copy()
y_testcopy = yTest.copy()

testScalerX = MinMaxScaler()
testScalerY = MinMaxScaler()

testcopy[numerics] =scalerX.transform(testcopy[numerics])
y_testcopy = testScalerY.fit_transform(y_testcopy.values.reshape(-1,1))


In [163]:
y_pred = RandomForest.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
print(mean_squared_error(y_testcopy,y_pred))
print(np.sqrt(mean_squared_error(y_testcopy,y_pred)))

0.029243667158783827
0.17100779853206644


In [164]:
y_pred = RandomForest.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
print(mean_squared_error(y_testcopy,y_pred))
print(np.sqrt(mean_squared_error(y_testcopy,y_pred)))

0.029243667158783827
0.17100779853206644


In [165]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x_train, y_train)

ypred = reg.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
sns.regplot(y_testcopy,ypred)
print(f"Linear Regression MSE : {mean_squared_error(y_testcopy,ypred)}")
print(f"Linear Regression RMSE : {np.sqrt(mean_squared_error(y_testcopy,ypred))}")


from sklearn.tree import DecisionTreeRegressor

dcs=DecisionTreeRegressor().fit(x_train,y_train)
ypred = dcs.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
print(f"Decision Regression MSE : {mean_squared_error(y_testcopy,ypred)}")
print(f"Decision Regression RMSE : {np.sqrt(mean_squared_error(y_testcopy,ypred))}")


from sklearn.svm import SVR

svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1).fit(x_train,y_train)
ypred = svr_rbf.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
print(f"SVR MSE : {mean_squared_error(y_testcopy,ypred)}")
print(f"SVR RMSE : {np.sqrt(mean_squared_error(y_testcopy,ypred))}")


from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=10).fit(x_train,y_train)
ypred = knn.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
print(f"KNN MSE : {mean_squared_error(y_testcopy,ypred)}")
print(f"KNN RMSE : {np.sqrt(mean_squared_error(y_testcopy,ypred))}")


from sklearn.ensemble import VotingRegressor

vote = VotingRegressor([('decesionTreeReg', dcs),("randomforest",RandomForestRegressor()),('svr',svr_rbf),('knn',knn)]).fit(x_train,y_train)
ypred = vote.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
print(f"Voting Regressor MSE : {mean_squared_error(y_testcopy,ypred)}")
print(f"Voting Regressor RMSE : {np.sqrt(mean_squared_error(y_testcopy,ypred))}")

ypred = model.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
print(f"DeepLearning  MSE : {mean_squared_error(y_testcopy,ypred)}")
print(f"DeepLearning  RMSE : {np.sqrt(mean_squared_error(y_testcopy,ypred))}")

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


Linear Regression MSE : 2305115.0403231215
Linear Regression RMSE : 1518.260531108914
Decision Regression MSE : 0.0554370511017982
Decision Regression RMSE : 0.23545074028721635
SVR MSE : 0.029732603576553104
SVR RMSE : 0.17243144602001428
KNN MSE : 0.040864489441264494
KNN RMSE : 0.2021496708908142
Voting Regressor MSE : 0.03701559011405254
Voting Regressor RMSE : 0.19239436092061674
DeepLearning  MSE : 0.039410257749743376
DeepLearning  RMSE : 0.19852016962954513


In [166]:
from sklearn.metrics import mean_absolute_error,mean_absolute_error,r2_score
def printTheOutput(model,modelName):
    ypred = model.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
    print(f"{modelName} Mean : {np.mean(ypred)} ")
    print(f"{modelName} Median : {np.median(ypred)}")
    print(f"{modelName} STD : {np.std(ypred)}")
    print(f"{modelName} MAE : {mean_absolute_error(y_testcopy,ypred)}")
    print(f"{modelName} MSE : {mean_squared_error(y_testcopy,ypred)}")
    print(f"{modelName} RMSE : {np.sqrt(mean_squared_error(y_testcopy,ypred))}")
    print()
printTheOutput(reg,"Regression")
printTheOutput(knn,"KNN")
printTheOutput(dcs,"Decision Tree")
printTheOutput(RandomForest,"Random Forest")
printTheOutput(vote,"Voting Classifier")
printTheOutput(model,"DeepLearning")

Regression Mean : 1220.1084520967868 
Regression Median : 1048.8917261168424
Regression STD : 903.8192329900746
Regression MAE : 1219.994754262897
Regression MSE : 2305115.0403231215
Regression RMSE : 1518.260531108914

KNN Mean : 0.2941083710140177 
KNN Median : 0.29571949075478765
KNN STD : 0.004238264212134062
KNN MAE : 0.19148773408788225
KNN MSE : 0.040864489441264494
KNN RMSE : 0.2021496708908142

Decision Tree Mean : 0.2692660040284733 
Decision Tree Median : 0.23038329969235619
Decision Tree STD : 0.15021854992802128
Decision Tree MAE : 0.18088448453744432
Decision Tree MSE : 0.0554370511017982
Decision Tree RMSE : 0.23545074028721635

Random Forest Mean : 0.2590568641456441 
Random Forest Median : 0.25738866469916555
Random Forest STD : 0.03724382780660746
Random Forest MAE : 0.15796770224977716
Random Forest MSE : 0.029243667158783827
Random Forest RMSE : 0.17100779853206644

Voting Classifier Mean : 0.27723719709106515 
Voting Classifier Median : 0.26808928951405886
Voting C

In [167]:
y_pred = RandomForest.predict(testcopy.drop(["author_id","publisher_id"],axis=1))
print(mean_squared_error(y_testcopy,y_pred))
print(np.sqrt(mean_squared_error(y_testcopy,y_pred)))
print((y_pred))

0.029243667158783827
0.17100779853206644
[0.24133297 0.23942584 0.22197462 0.2828056  0.24322687 0.20765457
 0.25285796 0.25475194 0.24388338 0.31361646 0.24571375 0.31361646
 0.25348188 0.18896052 0.31361646 0.25641508 0.26643668 0.28333549
 0.21294062 0.25958499 0.26139241 0.2828056  0.25702851 0.25489584
 0.25953819 0.20661851 0.23983236 0.24651991 0.26410242 0.24191601
 0.31361646 0.31361646 0.25320776 0.28282352 0.24738548 0.27817253
 0.21870673 0.25544615 0.28455365 0.31361646 0.31361646 0.24291052
 0.26519392 0.2363772  0.21588766 0.18959403 0.25733653 0.21313746
 0.31361646 0.2828056  0.25862579 0.26049234 0.17551083 0.26257935
 0.27353371 0.25146242 0.31361646 0.26238885 0.31341503 0.25741053
 0.31361646 0.22670037 0.24382385 0.25836531 0.25755493 0.31161181
 0.31361646 0.31361646 0.2585835  0.22333388 0.25137364 0.32810766
 0.26424901 0.31361646 0.24513008 0.23636123 0.31361646 0.2575593
 0.19016751 0.19474913 0.24379588 0.31361646 0.31361646 0.25904214
 0.25343364 0.25947221

In [168]:
print(testScalerY.inverse_transform(y_pred.reshape(-1,1)))

[[522069.1633098 ]
 [518056.46404353]
 [481338.25493325]
 [609329.55655578]
 [526054.0182617 ]
 [451208.1799251 ]
 [546318.28092447]
 [550303.30361342]
 [527435.33961303]
 [674157.09382341]
 [531286.52995272]
 [674157.09382341]
 [547631.03928901]
 [411875.00855661]
 [674157.09382341]
 [553802.64015148]
 [574888.56872127]
 [610444.47545606]
 [462330.28233418]
 [560472.28070372]
 [564275.17299624]
 [609329.55655578]
 [555093.31956628]
 [550606.07921671]
 [560373.80548181]
 [449028.258047  ]
 [518911.80411782]
 [532982.72104985]
 [569977.16141694]
 [523295.89541954]
 [674157.09382341]
 [674157.09382341]
 [547054.27527096]
 [609367.25395883]
 [534803.92287911]
 [599581.34988754]
 [474462.45434822]
 [551763.95954977]
 [613007.54868832]
 [674157.09382341]
 [674157.09382341]
 [525388.40203524]
 [572273.74184937]
 [511641.9704618 ]
 [468530.99396294]
 [413207.94156945]
 [555741.41581423]
 [462744.44508564]
 [674157.09382341]
 [609329.55655578]
 [558454.07652387]
 [562381.38445335]
 [383576.208

In [169]:
# Untuk melakukan prediksi gunakan cell ini model yang ini, perlu diperhatikan persamaan inputan kolom model dengan kolom data test

#ypred = RandomForest.predict(...))
#print(f"{"RandomForest"} Mean : {np.mean(ypred)} ")
#print(f"{"RandomForest"} Median : {np.median(ypred)}")
#print(f"{"RandomForest"} STD : {np.std(ypred)}")
#print(f"{"RandomForest"} MAE : {mean_absolute_error(...,ypred)}")
#print(f"{"RandomForest"} MSE : {mean_squared_error(...,ypred)}")
#print(f"{"RandomForest"} RMSE : {np.sqrt(mean_squared_error(...,ypred))}")
#print(testScalerY.inverse_transform(ypred.reshape(-1,1)))